In [71]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import requests
import json
import os
import numpy as np
import locale
import requests
%run Functions.ipynb #conecto con mi otro JN y utilizo mis funciones
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', None)
from datetime import datetime, timedelta

In [72]:
aemet = obtener_encabezado_api(api_file_path='../api.txt')

In [73]:
columnas_interes = ['fecha', 'provincia', 'tmed', 'prec', 'sol', 'tmin', 'tmax']

In [74]:
def obtener_url_datos(fecha_inicio, fecha_fin):
    base_url = 'https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/'
    parametros = f'fechaini/{fecha_inicio}/fechafin/{fecha_fin}/todasestaciones'
    url = f'{base_url}{parametros}'
    
    with open('../api.txt', 'r') as file:
        api_key = file.read().strip()


    # Hacer la solicitud GET
    response = requests.get(url, headers={'api_key': api_key})
    data = response.json()

    # Verificar el estado de la respuesta
    if data['estado'] == 200:
        return data['datos']
    else:
        print(f"Error en la solicitud: {data['descripcion']}")
        return None


In [75]:
import pandas as pd
import numpy as np

def procesar_mes(df_mes):
    # Reemplazar comas por puntos y convertir las columnas numéricas a tipo float
    df_mes['tmed'] = df_mes['tmed'].str.replace(',', '.').astype(float)
    df_mes['prec'] = df_mes['prec'].replace('Acum', np.nan)
    df_mes['prec'] = df_mes['prec'].str.replace(',', '.').replace('Ip', np.nan).astype(float)
    df_mes['sol'] = df_mes['sol'].str.replace(',', '.').astype(float)
    df_mes['tmin'] = df_mes['tmin'].str.replace(',', '.').astype(float)
    df_mes['tmax'] = df_mes['tmax'].str.replace(',', '.').astype(float)

    # Convertir la columna "fecha" en datetime y mostrar solo el mes y año
    df_mes['fecha'] = pd.to_datetime(df_mes['fecha']).dt.to_period('M')

    # Agrupar por provincia y calcular la media de cada columna
    df_media_mes = df_mes.groupby('provincia').agg({
        'fecha':'first',
        'tmed': 'mean',
        'prec': 'mean',
        'sol': 'mean',
        'tmin': 'mean',
        'tmax': 'mean'
    }).reset_index()

    # Renombrar las columnas para mayor claridad
    df_media_mes = df_media_mes.rename(columns={
        'fecha':'periodo',
        'tmed': 'media_tmed',
        'prec': 'media_prec',
        'sol': 'media_sol',
        'tmin': 'media_tmin',
        'tmax': 'media_tmax'
    })

    # Eliminar las filas correspondientes a 'CEUTA' y 'MELILLA'
    indices_a_eliminar = df_media_mes[df_media_mes['provincia'].isin(['CEUTA', 'MELILLA'])].index
    df_media_mes = df_media_mes.drop(indices_a_eliminar).reset_index(drop=True)

    # Capitalizar el nombre de las columnas
    df_media_mes.columns = df_media_mes.columns.str.capitalize()

    # Capitalizar las filas de la columna 'Provincia'
    df_media_mes['Provincia'] = df_media_mes['Provincia'].apply(lambda x: x.capitalize())

    return df_media_mes


In [76]:
import requests

def procesar_datos_mes(url, columnas_interes):
    # Obtener los datos de la API
    response = requests.get(url)

    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta JSON a un DataFrame
        datos = response.json()
        df = pd.DataFrame(datos)
        # Seleccionar columnas de interés
        df = df[columnas_interes]

        # Llamar a la función procesar_mes para realizar el procesamiento adicional
        df_procesado = procesar_mes(df)

        return df_procesado


In [78]:
# Lista de años a procesar (desde 2014 hasta 2023)
anos_a_procesar = list(range(2014, 2024))

# Columnas de interés para procesar en tu función procesar_datos_mes
columnas_interes = ['fecha', 'provincia', 'tmed', 'prec', 'sol', 'tmin', 'tmax']

# Llamada a la función principal
for ano in anos_a_procesar:
    for mes in range(1, 13):
        fecha_inicio = f'{ano}-{mes:02d}-01T00:00:00UTC'
        fecha_fin = f'{ano}-{mes:02d}-31T00:00:00UTC'  # Suponiendo siempre 31 días para simplificar

        url_datos = obtener_url_datos(fecha_inicio, fecha_fin)

        if url_datos:
            df_mes = procesar_datos_mes(url_datos, columnas_interes)
            # Puedes trabajar con el DataFrame df_mes según tus necesidades
            print(df_mes)


                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2014-01   10.067727    8.375463   1.601290   
1                Albacete  2014-01    7.683607    0.789655   4.319231   
2                Alicante  2014-01   12.108871    0.263415   5.108602   
3                 Almeria  2014-01   12.734416    0.703401   6.237097   
4             Araba/alava  2014-01    7.401613    4.291525   2.170968   
5                Asturias  2014-01    8.837582    5.075490   1.766512   
6                   Avila  2014-01    5.613978    2.415909   2.851613   
7                 Badajoz  2014-01   10.023596    1.661404   3.949020   
8               Barcelona  2014-01    9.839151    1.142925   3.749462   
9                 Bizkaia  2014-01   10.937903    6.613725   1.306452   
10                 Burgos  2014-01    5.682581    2.264474   1.803226   
11                Caceres  2014-01    9.320755    3.129193   4.088172   
12                  Cadiz  2014-01   12.102765    4

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2014-03   11.561304    2.472889   5.590196   
1                Albacete  2014-03   10.260833    0.413793   7.771429   
2                Alicante  2014-03   13.250806    0.095968   7.862366   
3                 Almeria  2014-03   14.132716    0.453947   8.903279   
4             Araba/alava  2014-03    8.533871    5.149153   5.562903   
5                Asturias  2014-03    9.471333    3.790785   5.150000   
6                   Avila  2014-03    7.089041    1.920238   7.080645   
7                 Badajoz  2014-03   12.237634    0.841622   7.622222   
8               Barcelona  2014-03   12.025352    0.480841   7.383871   
9                 Bizkaia  2014-03   11.368687    5.360769   4.390323   
10                 Burgos  2014-03    7.529677    1.591803   6.345968   
11                Caceres  2014-03   11.636264    1.504918   7.787097   
12                  Cadiz  2014-03   13.548357    1

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2014-05   14.216935    2.115447   7.304082   
1                Albacete  2014-05   17.520492    0.615966  10.800000   
2                Alicante  2014-05   18.398305    0.137288   8.528261   
3                 Almeria  2014-05   18.961143    0.182500  10.185246   
4             Araba/alava  2014-05   12.529032    1.171186   6.548387   
5                Asturias  2014-05   12.536364    2.778309   6.541038   
6                   Avila  2014-05   14.235632    1.008235   9.819355   
7                 Badajoz  2014-05   19.478495    0.408152  11.341290   
8               Barcelona  2014-05   16.653917    1.564953   7.833333   
9                 Bizkaia  2014-05   14.146774    1.679605   4.783871   
10                 Burgos  2014-05   12.105195    1.049593   8.262602   
11                Caceres  2014-05   18.400000    0.560215  11.009677   
12                  Cadiz  2014-05   19.474528    0

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2014-07   18.774368    2.005703   8.038776   
1                Albacete  2014-07   24.729060    0.244348  12.487931   
2                Alicante  2014-07   25.204839    0.000000  11.130108   
3                 Almeria  2014-07   24.199390    0.008284  10.150000   
4             Araba/alava  2014-07   19.125806    1.647368   7.083607   
5                Asturias  2014-07   17.910000    1.457551   6.490654   
6                   Avila  2014-07   20.438710    0.677778  10.864516   
7                 Badajoz  2014-07   25.189655    0.182967  12.046269   
8               Barcelona  2014-07   23.097653    1.801005   9.620879   
9                 Bizkaia  2014-07   19.868468    1.689286   5.109677   
10                 Burgos  2014-07   18.502667    0.845133   9.374138   
11                Caceres  2014-07   24.824138    0.589535  11.782222   
12                  Cadiz  2014-07   22.857547    0

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2014-09   19.543525    4.274419   6.641935   
1                Albacete  2014-09   22.148193    0.469136   8.605455   
2                Alicante  2014-09   24.509322    0.866667   7.598864   
3                 Almeria  2014-09   23.758140    0.755233   8.393548   
4             Araba/alava  2014-09   18.937097    2.126923   7.080645   
5                Asturias  2014-09   18.937860    3.159833   6.494180   
6                   Avila  2014-09   17.542529    1.011667   7.641935   
7                 Badajoz  2014-09   22.659538    1.747093   8.198592   
8               Barcelona  2014-09   22.354396    3.390110   5.908791   
9                 Bizkaia  2014-09   20.744954    0.819580   5.593548   
10                 Burgos  2014-09   18.121239    1.400000   7.207229   
11                Caceres  2014-09   21.610811    2.888415   8.022667   
12                  Cadiz  2014-09   22.253488    1

...

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2023-04   14.321577    1.722979   7.321978   
1                Albacete  2023-04   16.342276    0.052033  10.622034   
2                Alicante  2023-04   17.431452    0.059836  11.153763   
3                 Almeria  2023-04   18.610526    0.013158  10.512903   
4             Araba/alava  2023-04   11.730645    0.613115   7.514516   
5                Asturias  2023-04   12.870130    1.082623   7.300000   
6                   Avila  2023-04   15.085484    0.195161  10.706452   
7                 Badajoz  2023-04   18.807104    0.216129  11.125000   
8               Barcelona  2023-04   15.678226    0.592683   9.524731   
9                 Bizkaia  2023-04   13.579570    1.582114   6.374194   
10                 Burgos  2023-04   11.739869    0.520946   8.548889   
11                Caceres  2023-04   18.194194    0.198701  11.076344   
12                  Cadiz  2023-04   19.005093    0

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2023-06   19.212195    2.695154   8.505882   
1                Albacete  2023-06   22.060000    1.441905   9.171429   
2                Alicante  2023-06   23.764516    0.730081  11.133333   
3                 Almeria  2023-06   23.838095    0.212329  10.958065   
4             Araba/alava  2023-06   19.100000    3.053448   5.662903   
5                Asturias  2023-06   18.438387    2.748845   6.895699   
6                   Avila  2023-06   20.881818    2.027273   9.529032   
7                 Badajoz  2023-06   25.391209    0.570811  10.087919   
8               Barcelona  2023-06   23.289542    1.649664   8.492391   
9                 Bizkaia  2023-06   20.347312    3.042742   5.496774   
10                 Burgos  2023-06   18.880263    2.386667   7.869663   
11                Caceres  2023-06   24.614286    1.350000  10.490323   
12                  Cadiz  2023-06   24.232338    0

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2023-08   20.536842    0.934016   8.581818   
1                Albacete  2023-08   27.234959    0.042609  11.687719   
2                Alicante  2023-08   26.955645    0.197581  10.925581   
3                 Almeria  2023-08   27.509000    0.013740  11.490323   
4             Araba/alava  2023-08   21.232258    0.364407   8.583871   
5                Asturias  2023-08   19.653691    2.309831   6.646108   
6                   Avila  2023-08   24.388095    0.000000  12.170968   
7                 Badajoz  2023-08   28.320541    0.000000  11.575325   
8               Barcelona  2023-08   26.055844    1.446528   9.770968   
9                 Bizkaia  2023-08   21.501075    2.377049   6.232258   
10                 Burgos  2023-08   21.789032    0.049669  10.366304   
11                Caceres  2023-08   28.176129    0.001290  11.936559   
12                  Cadiz  2023-08   26.587500    0

                Provincia  Periodo  Media_tmed  Media_prec  Media_sol  \
0                A coruña  2023-10   17.531818    8.932636   4.590323   
1                Albacete  2023-10   18.203226    0.573770   6.730357   
2                Alicante  2023-10   20.523387    0.099194   8.523077   
3                 Almeria  2023-10   20.245528    0.407143   8.064516   
4             Araba/alava  2023-10   16.506452    3.208000   5.795161   
5                Asturias  2023-10   16.554513    4.927402   4.439583   
6                   Avila  2023-10   16.717742    1.912766   6.206452   
7                 Badajoz  2023-10   20.184946    3.962162   6.159091   
8               Barcelona  2023-10   20.556494    1.329752   6.645161   
9                 Bizkaia  2023-10   19.220325    2.707547   5.138710   
10                 Burgos  2023-10   15.468627    3.098675   5.382022   
11                Caceres  2023-10   19.495484    5.454605   5.452174   
12                  Cadiz  2023-10   21.070233    3

Error en la solicitud: No hay datos que satisfagan esos criterios


In [79]:
import pandas as pd

# Lista de años a procesar (desde 2014 hasta 2023)
anos_a_procesar = list(range(2014, 2024))

# Columnas de interés para procesar en tu función procesar_datos_mes
columnas_interes = ['fecha', 'provincia', 'tmed', 'prec', 'sol', 'tmin', 'tmax']

# Lista para almacenar los DataFrames resultantes
lista_dataframes = []

# Llamada a la función principal
for ano in anos_a_procesar:
    for mes in range(1, 13):
        fecha_inicio = f'{ano}-{mes:02d}-01T00:00:00UTC'
        fecha_fin = f'{ano}-{mes:02d}-31T00:00:00UTC'  # Suponiendo siempre 31 días para simplificar

        url_datos = obtener_url_datos(fecha_inicio, fecha_fin)

        if url_datos:
            df_mes = procesar_datos_mes(url_datos, columnas_interes)
            
            # Agregar el DataFrame a la lista
            lista_dataframes.append(df_mes)

# Concatenar todos los DataFrames en uno solo
df_final = pd.concat(lista_dataframes, ignore_index=True)

# Puedes trabajar con el DataFrame df_final según tus necesidades
df_final


Error en la solicitud: No hay datos que satisfagan esos criterios


,Provincia,Periodo,Media_tmed,Media_prec,Media_sol,Media_tmin,Media_tmax
0,A coruña,2014-01,10.067727,8.375463,1.601290,7.825909,12.305909
1,Albacete,2014-01,7.683607,0.789655,4.319231,3.475410,11.872131
2,Alicante,2014-01,12.108871,0.263415,5.108602,7.168548,17.044355
3,Almeria,2014-01,12.734416,0.703401,6.237097,8.529870,16.937662
4,Araba/alava,2014-01,7.401613,4.291525,2.170968,3.866129,10.941935
5,Asturias,2014-01,8.837582,5.075490,1.766512,5.835948,11.837255
6,Avila,2014-01,5.613978,2.415909,2.851613,1.632258,9.595699
7,Badajoz,2014-01,10.023596,1.661404,3.949020,6.155056,13.895506
8,Barcelona,2014-01,9.839151,1.142925,3.749462,5.689623,13.991981
9,Bizkaia,2014-01,10.937903,6.613725,1.306452,7.880645,14.000806


In [81]:
df_final[50:100]

,Provincia,Periodo,Media_tmed,Media_prec,Media_sol,Media_tmin,Media_tmax
50,A coruña,2014-02,9.557014,8.672768,2.766443,6.594118,12.522624
51,Albacete,2014-02,8.005833,0.845045,5.741071,3.709167,12.294167
52,Alicante,2014-02,12.552459,0.086667,5.587778,7.213115,17.885246
53,Almeria,2014-02,13.030719,0.674172,7.016129,8.811765,17.250980
54,Araba/alava,2014-02,6.579032,5.212727,4.050000,1.512903,11.635484
55,Asturias,2014-02,8.470307,5.055556,3.746729,4.605802,12.336860
56,Avila,2014-02,4.597778,2.614607,4.303226,0.206667,8.995556
57,Badajoz,2014-02,9.796988,2.997814,3.456954,5.842169,13.748193
58,Barcelona,2014-02,10.220792,1.064216,5.303226,5.531188,14.916337
59,Bizkaia,2014-02,9.898374,6.222876,2.493548,5.647154,14.145528
